In [ ]:
!pip install peft

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.9 MB/s eta 0:00:00


In [ ]:
!pip install -U accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1


In [ ]:
# Import the necessary packages
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftConfig, PeftModel
import bitsandbytes as bnb


In [ ]:
# Connect with the model from huggingface
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
adapter = "GRMenon/mental-health-mistral-7b-instructv0.2-finetuned-V2"

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    add_bos_token=True,
    trust_remote_code=True,
    padding_side='left'
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Load the configuration from the pretrained adapter
config = PeftConfig.from_pretrained(adapter)

# Define the quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    quant_type="nf4",
    bnb_optim_bits=32
)

# Load the base model with the quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map='auto',
    torch_dtype='auto'
)

# Load the PeftModel with the fine-tuned adapter
model = PeftModel.from_pretrained(model, adapter)

# Move the model to the appropriate device (GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

Unused kwargs: ['quant_type', 'bnb_optim_bits']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k

In [ ]:
# Prompt content:
messages = [
    {
        "role": "user", "content": "Hey Connor! I have been feeling a bit down lately. I could really use some advice on how to feel better?"
    }
]

input_ids = tokenizer.apply_chat_template(conversation=messages,
                                          tokenize=True,
                                          add_generation_prompt=True,
                                          return_tensors='pt').to(device)


output_ids = model.generate(input_ids=input_ids, max_new_tokens=512, do_sample=True, pad_token_id=2)
response = tokenizer.batch_decode(output_ids.detach().cpu().numpy(), skip_special_tokens = True)

# Model response:
print(response[0])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[INST] Hey Connor! I have been feeling a bit down lately. I could really use some advice on how to feel better? [/INST] I'm so glad you reached out. I can provide you with some general information and the type of people and things that I find uplifting.

First, everyone feels down, sad, or in the dumps. It's normal and normal to try and get back to your normal, when you are ready. So, don't panic that this is the only time you have ever felt like this. But, feeling down is never good, and I agree that if this is more than what you usually feel or, heaven forbid, a change, then do something.

A lot of people find things that they like to do, when they are feeling down, as a distraction such as:
- Take a bath, or shower for at least 10 minutes
- Read
- Write, in a journal
- Call someone you like, or that you trust
- Text, DM, or chat with someone
- Go for a walk or a run, and when you are thinking of the person's company, then try to focus on the exercise
- Take yourself out for lunch, o

In [ ]:
messages = [
    {
        "role": "user", "content": " i am feeling down, i feel like i want to die"
    }
]

input_ids = tokenizer.apply_chat_template(conversation=messages,
                                          tokenize=True,
                                          add_generation_prompt=True,
                                          return_tensors='pt').to(device)


output_ids = model.generate(input_ids=input_ids, max_new_tokens=512, do_sample=True, pad_token_id=2)
response = tokenizer.batch_decode(output_ids.detach().cpu().numpy(), skip_special_tokens = True)

# Model response:
print(response[0])

[INST]  i am feeling down, i feel like i want to die [/INST] I am so sorry you're feeling so badly. There is hope though. It doesn't the way you're feeling right now. There are many who feel the same. So, you are alone in this it is true, but that's not a reason to give up. I do not know you, but I can certainly empath The right  counselor could mean the world to you.

  There are many therapists that specialize in helping people to feel better. This means that they have taken classes  and  done additional training on top of their  regular training on how to help people feel better when they are depressed.  The right counselor may  be the  answer to  your  feelings of hopelessness.  If you are looking  for an  in- person  counselor,   then  you should  call   the  local mental health association in your area and  ask for one that specializes  in depression. If  you  are looking  for an online therapist,  start  by  looking  for something that  sounds  close to what  you  are  going  th

In [1]:
import pandas as pd

df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df['C'] = df['A'] + df['B']
print(df['C'].sum())

21


In [2]:
from sklearn.preprocessing import StandardScaler
import numpy as np

data = np.array([[1, 2], [3, 4], [5, 6]])
scaler = StandardScaler()
scaler.fit(data)
print(scaler.transform([[3,4]]))

[[0. 0.]]


In [4]:
A = [1,3,5,7,9]
B = A
B[0] = -999
print(A[0])

-999
